In [48]:
#PASSO 1: Tokenização
#Aqui a gente transforma o texto em números

import numpy as np

#Aqui abaixo na variável texto precisa fazer a injeção de textos das fontes que quiser.
texto = "DOCUMENTO MOCK\nCapítulo 1: Introdução  \nEste é um texto fictício com acentos (á, é, ã, ç) e símbolos (@#%) \nUse para testar seu tokenizador e embeddings!\nAutor: Você."
texto = texto.split(' ')
vocabulario = sorted(list(set(texto)))
print(vocabulario)

['', '\nEste', '\nUse', '(@#%)', '(á,', '1:', 'DOCUMENTO', 'Introdução', 'MOCK\nCapítulo', 'Você.', 'acentos', 'com', 'e', 'embeddings!\nAutor:', 'fictício', 'para', 'seu', 'símbolos', 'testar', 'texto', 'tokenizador', 'um', 'ã,', 'ç)', 'é', 'é,']


In [49]:
#Seta cada palavra a um token e vice versa
token_para_id = {c: i for i, c in enumerate (vocabulario)}
id_para_token = {i: c for i, c in enumerate (vocabulario)}
print(token_para_id)
print(id_para_token)


{'': 0, '\nEste': 1, '\nUse': 2, '(@#%)': 3, '(á,': 4, '1:': 5, 'DOCUMENTO': 6, 'Introdução': 7, 'MOCK\nCapítulo': 8, 'Você.': 9, 'acentos': 10, 'com': 11, 'e': 12, 'embeddings!\nAutor:': 13, 'fictício': 14, 'para': 15, 'seu': 16, 'símbolos': 17, 'testar': 18, 'texto': 19, 'tokenizador': 20, 'um': 21, 'ã,': 22, 'ç)': 23, 'é': 24, 'é,': 25}
{0: '', 1: '\nEste', 2: '\nUse', 3: '(@#%)', 4: '(á,', 5: '1:', 6: 'DOCUMENTO', 7: 'Introdução', 8: 'MOCK\nCapítulo', 9: 'Você.', 10: 'acentos', 11: 'com', 12: 'e', 13: 'embeddings!\nAutor:', 14: 'fictício', 15: 'para', 16: 'seu', 17: 'símbolos', 18: 'testar', 19: 'texto', 20: 'tokenizador', 21: 'um', 22: 'ã,', 23: 'ç)', 24: 'é', 25: 'é,'}


In [50]:
#Tokenização
tokens = [token_para_id[c] for c in texto] #Reescreve o texto inicial como tokens
print("Tokens:", tokens)

Tokens: [6, 8, 5, 7, 0, 1, 24, 21, 19, 14, 11, 10, 4, 25, 22, 23, 12, 17, 3, 2, 15, 18, 16, 20, 12, 13, 9]


In [51]:
#PASSO 2: Embedding (Vetores de Palavras)
#Cada token vira um vetor de dimensão "d_model"

d_model = 256 #Isso é o tanto de números que vão representar uma letra! (Sim, precisa ser um número razoavelmente alto porque precisa diferenciar acentuações, maiúsculas, minúsculas e todo tipo de variação que existir de um caractere para outro)
vocab_size = len(vocabulario)

#Embeddings aleatórios (normalmente aprendidos durante o treino)
embedding = np.random.randn(vocab_size, d_model)

#Transforma tokens em vetores
token_embeddings = embedding[tokens] #Shape: [len(tokens), d_model]
print(token_embeddings)

[[-0.01786658  1.91531678 -0.84834873 ...  2.49264046 -1.91417195
  -0.54391071]
 [-0.16182209  0.49858315  2.23865579 ... -2.71723215  1.21656818
  -0.37567252]
 [-0.83725326  1.04301379  1.03118608 ... -0.57616258 -1.24966438
   0.35365948]
 ...
 [-1.1016822   1.31416071 -0.67419309 ...  1.68009913 -0.36395967
   0.42462353]
 [-0.0525345  -0.59194368 -0.60421231 ...  1.15227626  0.07397107
  -1.80672398]
 [ 0.02892368 -0.02983188  0.9794713  ... -1.66929551 -0.25618648
   1.19562879]]


In [52]:
#PASSO 3: Mecanismo de Atenção (Self-Attention)
#Calcula a relação entre tokens, versão simplificada:
def attention(Q, K, V):
    # Q, K, V são matrizes de consulta, chave e valor
    d_k = Q.shape[-1]
    scores = np.dot(Q, K.T) / np.sqrt(d_k)  # Similaridade: Calcula o "quão bem" a palavra Q se relaciona com as outras (K)
    weights = np.exp(scores) / np.sum(np.exp(scores), axis=-1, keepdims=True)  # Softmax: Transforma os scores em probabilidades (soma = 1)
    return np.dot(weights, V)  # Combinação: Combina os embeddings das palavras (V) usando os pesos de atenção

# Exemplo:
Q = token_embeddings  # Consulta: O que cada palavra está buscando
K = token_embeddings  # Chave: Como cada palavra se descreve
V = token_embeddings  # Valor: Informação real de cada palavra
saida_attention = attention(Q, K, V)


In [53]:
#PASSO 4: Rede Neural Feed-Forward
#Uma MLP simples para processar os vetores após o mecanismo de atenção
def feed_forward(x, W1, b1, W2, b2):
    hidden = np.maximum(0, np.dot(x, W1) + b1)  # Camada oculta com ReLU
    return np.dot(hidden, W2) + b2 # Camada de saída

# Pesos aleatórios (em um modelo real, seriam aprendidos)
W1 = np.random.randn(d_model, 4 * d_model) # Pesos da camada oculta
b1 = np.random.randn(4 * d_model) # Vieses da camada oculta
W2 = np.random.randn(4 * d_model, d_model) # Pesos da camada de saída
b2 = np.random.randn(d_model) # Vieses da camada de saída

saida_ff = feed_forward(saida_attention, W1, b1, W2, b2)

print(saida_attention)

[[-0.01786511  1.9153097  -0.84834456 ...  2.49263048 -1.91416282
  -0.54390648]
 [-0.16181902  0.49858038  2.23864546 ... -2.71721724  1.21656102
  -0.37566793]
 [-0.83724962  1.04301106  1.03118314 ... -0.57616014 -1.24965876
   0.35365962]
 ...
 [-1.10167998  1.31415928 -0.67419174 ...  1.68009728 -0.36395892
   0.42462308]
 [-0.05252838 -0.59193978 -0.60420591 ...  1.15226988  0.07397181
  -1.80670164]
 [ 0.02892589 -0.02982777  0.97946472 ... -1.66927893 -0.25618466
   1.1956216 ]]


In [54]:
#PASSO 5: Treinamento
#Para treinar precisaríamos
#1 - Dados: Um corpus de texto (ex.: livros do Projeto Gutenberg).
#2 - Função de perda (Loss Function): Cross-entropy entre previsões e tokens reais.
#3 - Backpropagation: Implementar gradientes manualmente (NumPy não tem autograd).

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True)) # Evita overflow
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

def cross_entropy(y_pred, y_true):
    m = y_true.shape[0] #Número de exemplos
    log_probs = -np.log(y_pred[range(m), y_true]) # Penaliza previsões erradas
    return np.sum(log_probs) / m # Média do erro

# Exemplo fictício:
y_pred = softmax(np.random.randn(len(tokens), vocab_size))  # Previsões aleatórias
y_true = np.array(tokens)  # Tokens reais
loss = cross_entropy(y_pred, y_true)
print("Loss:", loss) # Quanto menor, melhor

Loss: 3.653326188796029


In [ ]:
#JUNTANDO TODOS OS PASSOS E PREVENDO A PRÓXIMA LETRA
import numpy as np

# Configurações
texto = "ola mundo"
vocabulario = sorted(list(set(texto)))
token_para_id = {c: i for i, c in enumerate(vocabulario)}
id_para_token = {i: c for i, c in enumerate(vocabulario)}
d_model = 16  # Dimensão dos embeddings

# 1. Tokenização (input do usuário)
input_texto = "ola mun"  # Tente prever o próximo caractere
tokens = [token_para_id[c] for c in input_texto]

# 2. Embeddings (aleatórios, pois não treinamos)
embedding = np.random.randn(len(vocabulario), d_model)
token_embeddings = embedding[tokens]  # Shape: [len(tokens), d_model]

# 3. Self-Attention (simplificada)
def attention(Q, K, V):
    d_k = Q.shape[-1]
    scores = np.dot(Q, K.T) / np.sqrt(d_k)
    weights = np.exp(scores) / np.sum(np.exp(scores), axis=-1, keepdims=True)
    return np.dot(weights, V)

Q = token_embeddings
K = token_embeddings
V = token_embeddings
saida_attention = attention(Q, K, V)

# 4. Feed-Forward (aleatório)
W1 = np.random.randn(d_model, 4 * d_model)
b1 = np.random.randn(4 * d_model)
W2 = np.random.randn(4 * d_model, d_model)
b2 = np.random.randn(d_model)

def relu(x):
    return np.maximum(0, x)

hidden = relu(np.dot(saida_attention[-1], W1) + b1)  # Pega o último token
logits = np.dot(hidden, W2) + b2  # Shape: [d_model]

# 5. Predição do próximo token
probs = np.exp(logits) / np.sum(np.exp(logits))  # Softmax
proximo_token_id = np.argmax(probs)
proximo_char = id_para_token[proximo_token_id]

print(f"Input: '{input_texto}' → Próximo caractere previsto: '{proximo_char}'")